In [31]:
import sys
import os

chemin_actuel = os.getcwd()
chemin_parent = os.path.dirname(chemin_actuel)
sys.path.append(chemin_parent)

from class_sportif import *
from algo_matching_sportifs import *

Sportif()

Sportif(nom=None, prenom=None, sexe=None, age=None, nationalite=None, localisation=None, distance_rencontre=None, niveau_sports=None, attentes=None, genre_recherche=None, min_age_recherchee=None, max_age_recherchee=None, nb_swipes_restants=None, photo_profil=None)

In [32]:
def func_test_calculer_distance():
    
    # Définition d'une tolérance pour les comparaisons en virgule flottante
    TOLERANCE = 0.01 # Tolérance de 10 mètres pour la distance en km

    print("--- 🧪 Début des Tests Manuels de Distance Haversine ---")

    # ==========================================================
    ## 🎯 SCÉNARIO 1 : Distance nulle (Même point)
    # ==========================================================
    print("\n[TEST 1] : Même Point")
    
    # Coordonnées de l'Arc de Triomphe
    lat_arc = 48.8738, 2.2950
    
    distance = func_calculer_distance(lat_arc[0], lat_arc[1], lat_arc[0], lat_arc[1])
    
    # Vérification
    if abs(distance - 0.0) < TOLERANCE:
        print(f"✅ Réussi : Distance calculée : {distance:.4f} km. (Attendu : ~0)")
    else:
        print(f"❌ Échoué : Distance calculée : {distance:.4f} km. (Attendu : 0)")


    # ==========================================================
    ## 🎯 SCÉNARIO 2 : Petite Distance (Paris - Montmartre)
    # Distance de référence Google Maps : ~4.2 km
    # ==========================================================
    print("\n[TEST 2] : Petite Distance (Arc de Triomphe à Montmartre)")

    lat_montmartre = 48.8867, 2.3431
    reference_distance = 3.7  # km

    distance = func_calculer_distance(lat_arc[0], lat_arc[1], lat_montmartre[0], lat_montmartre[1])

    # Vérification
    if abs(distance - reference_distance) < 0.2: # Tolérance élargie à 200m
        print(f"✅ Réussi : Distance calculée : {distance:.2f} km. (Référence : {reference_distance} km)")
    else:
        print(f"❌ Échoué : Distance calculée : {distance:.2f} km. (Référence : {reference_distance} km)")


    # ==========================================================
    ## 🎯 SCÉNARIO 3 : Grande Distance (Paris - New York)
    # Distance de référence Google Maps : ~5837 km (selon la formule Haversine, cette valeur est très précise)
    # ==========================================================
    print("\n[TEST 3] : Grande Distance (Paris à New York)")
    
    lat_ny = 40.7128, -74.0060 # Lower Manhattan
    reference_distance = 5837.2 # km (Distance orthodromique)

    distance = func_calculer_distance(paris_lat, paris_lon, lat_ny[0], lat_ny[1])

    # Vérification (Tolérance de 1 km sur une grande distance)
    if abs(distance - reference_distance) < 1.0:
        print(f"✅ Réussi : Distance calculée : {distance:.2f} km. (Référence : {reference_distance} km)")
    else:
        print(f"❌ Échoué : Distance calculée : {distance:.2f} km. (Référence : {reference_distance} km)")
        
    print("\n--- ✅ Fin des Tests ---")

# Définition des coordonnées de base
paris_lat, paris_lon = 48.8566, 2.3522 # Notre-Dame

# Exécution
func_test_calculer_distance()

--- 🧪 Début des Tests Manuels de Distance Haversine ---

[TEST 1] : Même Point
✅ Réussi : Distance calculée : 0.0000 km. (Attendu : ~0)

[TEST 2] : Petite Distance (Arc de Triomphe à Montmartre)
✅ Réussi : Distance calculée : 3.80 km. (Référence : 3.7 km)

[TEST 3] : Grande Distance (Paris à New York)
✅ Réussi : Distance calculée : 5837.24 km. (Référence : 5837.2 km)

--- ✅ Fin des Tests ---


In [33]:
def func_test_matching_final():
    nom_fichier = 'data.json'
    
    # --- 1. SPORTIF TEST ---
    sportif_a = Sportif(
        nom="Martin",
        prenom="Bob",
        sexe="Homme",
        age=30,
        nationalite="Française",
        localisation=[43.6050, 1.4448],
        distance_rencontre=100,
        niveau_sports={"Running": 4},
        attentes="Performance",
        genre_recherche="Femme",
        min_age_recherchee=25,
        max_age_recherchee=35,
        photo_profil="bytes_dummy_bob",
        nb_swipes_restants=10
    )

    # --- 2. DONNÉES D’ENTRÉE JSON ---
    donnees_json = [
  {
    "nom": "Durand",
    "prenom": "Julien",
    "sexe": "Homme",
    "age": 29,
    "nationalite": "Française",
    "localisation": [45.7578, 4.832],
    "distance_rencontre": 15,
    "niveau_sports": {"Escalade": 4, "Randonnée": 5},
    "attentes": "amitié",
    "genre_recherche": "Femme",
    "min_age_recherchee": 25,
    "max_age_recherchee": 35,
    "photo_profil": "uploads/test_profil.png",
    "nb_swipes_restants": 50
  },
  {
    "nom": "Leroy",
    "prenom": "Amélie",
    "sexe": "Femme",
    "age": 27,
    "nationalite": "Française",
    "localisation": [43.6047, 1.4442],
    "distance_rencontre": 100,
    "niveau_sports": {"Badminton": 3, "Yoga": 4},
    "attentes": "amour",
    "genre_recherche": "Homme",
    "min_age_recherchee": 20,
    "max_age_recherchee": 30,
    "photo_profil": "bytes_dummy_amelie",
    "nb_swipes_restants": 15
  },
  {
    "nom": "Petit",
    "prenom": "Marc",
    "sexe": "Homme",
    "age": 41,
    "nationalite": "Française",
    "localisation": [48.8566, 2.3522],
    "distance_rencontre": 10,
    "niveau_sports": {"Natation": 5, "Course à pied": 3},
    "attentes": "amitié",
    "genre_recherche": "Femme",
    "min_age_recherchee": 35,
    "max_age_recherchee": 45,
    "photo_profil": "bytes_dummy_marc",
    "nb_swipes_restants": 100
  },
  {
    "nom": "Bernard",
    "prenom": "Sophie",
    "sexe": "Femme",
    "age": 34,
    "nationalite": "Française",
    "localisation": [43.2965, 5.3698],
    "distance_rencontre": 35,
    "niveau_sports": {"Planche à voile": 4, "Kitesurf": 5},
    "attentes": "amour",
    "genre_recherche": "Peu importe",
    "min_age_recherchee": 30,
    "max_age_recherchee": 40,
    "photo_profil": "bytes_dummy_sophie",
    "nb_swipes_restants": 5
  },
  {
    "nom": "Moreau",
    "prenom": "Lucas",
    "sexe": "Homme",
    "age": 20,
    "nationalite": "Française",
    "localisation": [47.2184, -1.5536],
    "distance_rencontre": 20,
    "niveau_sports": {"Basket-ball": 4, "Musculation": 1},
    "attentes": "amour",
    "genre_recherche": "Femme",
    "min_age_recherchee": 18,
    "max_age_recherchee": 25,
    "photo_profil": "bytes_dummy_lucas",
    "nb_swipes_restants": 30
  },
  {
    "nom": "Dupond",
    "prenom": "Pierre",
    "sexe": "Homme",
    "age": 30,
    "nationalite": "Française",
    "localisation": [48.8, 2.3],
    "distance_rencontre": 20,
    "niveau_sports": {"Judo": 5},
    "attentes": "amour",
    "genre_recherche": "Femme",
    "min_age_recherchee": 25,
    "max_age_recherchee": 35,
    "photo_profil": "bytes_dummy_replaced",
    "nb_swipes_restants": 0
  }
]




    # --- 3. Sauvegarde JSON ---
    with open(nom_fichier, 'w', encoding='utf-8') as f:
        json.dump(donnees_json, f, indent=2, ensure_ascii=False)

    print("\n--- 🧪 Début des Tests de Matching (Version Finale) ---")

    # --- 4. TEST PRINCIPAL ---
    resultats = func_algo_matching(sportif_a)
    prenoms_trouves = [s.prenom for s in resultats]
    prenoms_attendus = ["Amélie"]  # ⚠️ à adapter selon tes données réelles

    print("\n--- Analyse des Résultats ---")
    if set(prenoms_trouves) == set(prenoms_attendus):
        print(f"✅ Réussi (Principal) : Matchs trouvés = {prenoms_trouves}")
    else:
        print(f"❌ Échoué (Principal) : Trouvés = {prenoms_trouves}, Attendus = {prenoms_attendus}")

    # --- 5. TEST NB SWIPES ---
    sportif_echec_swipes = Sportif(
        nom="Gaspard", prenom="Gaspard", sexe="Homme", age=30,
        nationalite="Français", localisation=[0, 0], distance_rencontre=10,
        attentes="Loisir", genre_recherche="Femme",
        min_age_recherchee=20, max_age_recherchee=40, nb_swipes_restants=0,
        photo_profil=None
    )
    resultats_swipes = func_algo_matching(sportif_echec_swipes)
    if len(resultats_swipes) == 0:
        print("✅ Réussi (Swipes) : Le matching s'arrête quand les swipes sont épuisés.")
    else:
        print("❌ Échoué (Swipes) : Le matching continue malgré nb_swipes_restants = 0.")

    # --- 6. Nettoyage ---
    if os.path.exists(nom_fichier):
        os.remove(nom_fichier)

# Exécution du test
func_test_matching_final()


--- 🧪 Début des Tests de Matching (Version Finale) ---

--- Analyse des Résultats ---
✅ Réussi (Principal) : Matchs trouvés = ['Amélie']
✅ Réussi (Swipes) : Le matching s'arrête quand les swipes sont épuisés.
